In [1]:
import numpy as np
import matplotlib.pyplot as plt

$$\hat{Z}_{k+1}(w)=\int\left( \underset{i}{\sum}\underset{j}{\sum}C_{p_0p_0}(w,w').G_k^i(w').(S^{-1})^{ij}.\{d_o^j - g^j(\hat{Z}_{k}) + \int G_k^j(w^{''}).Z_{k}(w^{''}) dw^{''} \}\right)dw'$$

where $$C_{p_0p_0}(w,w')= \sigma^2 \text{exp} \left\{ -\dfrac{1}{2}\dfrac{(w-w')^2}{\Delta^2}\right\},$$
$$G_k^i(w)= \dfrac{-2(H - z_k(w))}{(x^i - w)^2 + [H - z_k(w)]^2}$$ and 
$$S_k^{ij}=(C_{d_0d_0})^{ij} + \int \left( \int G_k^i(w').C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'$$

In [ ]:
sig=
H=
d0=
C_d0_d0=
m=         #for  x_i
n=         #for  w_i
a=
b=
nb_iter=
delta=

In [ ]:
hx=(b-a)/(m-1)
hw=(b-a)/(n-1)

W = np.zeros(n)
W[0]=a
W[-1]=b

for i in range(1,n):
    X[i]= X[i-1] + hw

X = np.zeros(m)
X[0]=a
X[-1]=b

for i in range(1,m):
    X[i]= X[i-1] + hx

In [ ]:
def C_p0_p0(w,w_prime,sigma,Delta):
    
    arg=-1/2 *((w-w_prime)**2)/(Delta**2)
    
    return sigma**2 * np.exp(arg)


def G_k(w,x_i,Zk_w,H):
    
    return -2*(H-Z_w)/((x_i-w)**2 - (H-Zk_w)**2)


Computation of
$$S_k^{ij}=(C_{d_0d_0})^{ij} + \int \left( \int G_k^i(w').C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'=(C_{d_0d_0})^{ij} + \int G_k^i(w') \left( \int C_{p_0p_0}(w',w^{''}).G_k^j(w{''})  dw^{''}\right) dw'$$

In [ ]:
def S_k_ij(G_k,C_d0_d0,W,X,Zk_w,H,i,j):
    

In [4]:
for i in range(1,5):
    print(i)

1
2
3
4
